### Name: SALONI K SHAH
### Hopkins ID: 9A3AE4
### Date: 27th September 2025

#                              **PROBLEM SET 2 - MACHINE LEARNING IN STATISTICS** 

### My Understanding of the Assignment

I must refer to the California Housing Dataset and the end-to-end machine learning project that the professor taught us in Week 2 and 3 of this course, and perform the tasks assigned for this problem set. 
Specifically, in this assignment, I must load the data, train and test the dataset on two models:
1) Model 1 which uses stratified sampling - i.e., divide the dataset into four stratas/quartiles using the variable median_income' and ocean_proximity. This stratfication gives me a new auxilary variable which has 'ocean_proximity and income quartile'. I must then use this variable to split the dataset into training and testing and perform a regression.
2) Model 2 which uses random sampling strategy to split the dataset into training and testing and then I must perform regression.
Once I have the results of the two models, I then need to compare and evaluate the performance.

For this assignment, I have refered to the Week 2 codes that the instructor had taught in the class. However, it seems that the codes for loading the data are from the previous edition of Geron's textbook. Therefore, I referred to the online version of Geron's Third Edition Textbook - Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, and followed the instructions of loading the data. The updated dataset and codes have been provided by the author here - https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb. I have used all my codes refering as provided in this link. 

**Note: I did not use any AI tools for my assignment. All my codes are from the link I shared.** 

### STEP 1: Set up

In [1]:
#load Python's system module
#use assert to stop execution if Python is too old. Instead can also use 'if' and else' command
import sys             
assert sys.version_info >= (3, 7)

#import ski-kit learn and check its version
import sklearn
assert sklearn.__version__ >= "0.20"

#numpy aliased to np is the main library for array/math
#os gives portable file-path and filesystem utilities
import numpy as np
import os

#%matplotlib is Jupiter's magic command to create all graphs and plots within the notebook 
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc ('axes',labelsize=14)
mpl.rc ('xtick',labelsize=12)
mpl.rc ('ytick',labelsize=12)

#PROJECT_ROOT_DIR helps to create a folder for your project. That is, the path to the folder on the computer
#CHAPTER_ID helps to give a name of the subfolder
#IMAGES_PATH creates a path to store images combining the path of the project folder and sub folder
#the final code creates the folders if not created. 
PROJECT_ROOT_DIR = "/Users/salonikshah/Desktop/0. MS Applied Economics/8. Machine Learning in Statistics/Week 1 & 2 _California Housing Project"
CHAPTER_ID = "California Housing"
IMAGES_PATH = os.path.join (PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs (IMAGES_PATH, exist_ok = True)

#fig_id is the filename stem (no extension)
#Constructs path inside IMAGES_PATH, e.g. ./images/California Housing/my_plot.png
#Prints a small log message so you see what’s being saved
#plt.tight_layout() auto-adjusts subplot spacing to avoid clipped labels(this is optional)
#plt.savefig(...) writes the current figure to disk as a high-resolution PNG (dpi=300 good for docs).
def save_fig(fig_id, tight_layout=True):
    path = os.path.join (IMAGES_PATH, fig_id + ".png")
    print("saving_figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format="png", dpi=300)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### STEP 2: Get the data

In [2]:
#imports pandas as 'pd' to work with tabular data or DataFrames (refer to similar ones before for matlibplot)
#imports and extracts compressed .tgz files
#urllib.request is used to download files from the internet using the defined url
#allows us to treat raw bytes in memory as a file-like object (needed because Pandas expects something “file-like”).

import pandas as pd
import tarfile 
import urllib.request
import io

#creates a reusable function that downloads, extracts, and loads the housing dataset
#insert the url or location of the compressed dataset on Github
#response is used for the stream of bytes you get back from the internet. 'urllib.request.urlopen(url) opens a connection to the URL and starts downloading the file
#io.Bytes10(...) wraps the raw bytes into a seekable file like object, so libraries like tarfile can work with it. '.read' downloads the full '.tgz' file into memory
#opens the .tgz archive sitting in the memory. with...as... is used to open and close the tarball when done
#csv_file(...) extracts only the .csv file from the tarball
#.read() reads the .csv file 
#io.BytesIO() wraps that content into another file-like object that Pandas can read.

def load_housing_data():
    url = "https://github.com/ageron/data/raw/main/housing.tgz"
    response = urllib.request.urlopen(url)
    file_like_object = io.BytesIO(response.read())
    with tarfile.open(fileobj = file_like_object) as housing_tarball:
        csv_file = housing_tarball.extractfile("housing/housing.csv")
        csv_data = io.BytesIO(csv_file.read())
    return pd.read_csv(csv_data)

#store the housing dataset in the variable 'housing'
housing = load_housing_data()

### STEP 3: Take a look at the data

In [3]:
housing.head(15)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
housing["ocean_proximity"].value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [6]:
housing ["median_income"].value_counts()

median_income
3.1250     49
15.0001    49
2.8750     46
4.1250     44
2.6250     44
           ..
4.2670      1
2.1217      1
4.9706      1
3.4450      1
2.0943      1
Name: count, Length: 12928, dtype: int64

### STEP 4: Create income quartiles 

In [7]:
housing["income_quartile"] = pd.cut(
    housing["median_income"],
    bins=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)

In [8]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### STEP 5: SplitCreate a new auxiliary variable that combines ocean proximity with income quartile 

In [9]:
housing["strat_col"] = housing["ocean_proximity"].astype(str) + "_" + housing["income_quartile"].astype(str)

In [10]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,income_quartile,strat_col
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,Q3,NEAR BAY_Q3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,Q3,NEAR BAY_Q3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,Q2,NEAR BAY_Q2
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,Q2,NEAR BAY_Q2
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,Q1,NEAR BAY_Q1


In [15]:
#I want to see sample data of the new auxilary variable created and number of rows that fall under each category/object of the variable. 

housing[["ocean_proximity", "income_quartile", "strat_col"]].head()

,ocean_proximity,income_quartile,strat_col
0,NEAR BAY,Q3,NEAR BAY_Q3
1,NEAR BAY,Q3,NEAR BAY_Q3
2,NEAR BAY,Q2,NEAR BAY_Q2
3,NEAR BAY,Q2,NEAR BAY_Q2
4,NEAR BAY,Q1,NEAR BAY_Q1


In [16]:
housing["strat_col"].value_counts()

strat_col
INLAND_Q1        5138
<1H OCEAN_Q1     5077
<1H OCEAN_Q2     3571
NEAR OCEAN_Q1    1623
INLAND_Q2        1346
NEAR BAY_Q1      1324
NEAR OCEAN_Q2     908
NEAR BAY_Q2       840
<1H OCEAN_Q3      409
NEAR BAY_Q3       103
NEAR OCEAN_Q3     102
<1H OCEAN_Q4       79
INLAND_Q3          60
NEAR OCEAN_Q4      25
NEAR BAY_Q4        23
INLAND_Q4           7
ISLAND_Q1           5
Name: count, dtype: int64

## STEP 6: Use StratifiedShuffleSplit to divide the dataset into training and testing sets, stratifying by the auxiliary variable

In [17]:
from sklearn.model_selection import StratifiedShuffleSplit

In [19]:
# Code a commande split: 80/20 with a single splint. Random generate is 42. 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Create two datasets by splitting the housing data stratified by housing["strat_col"]. 
#Note that this is the same code as taught by the professor in the class, but replaced ["income_cat"] by ["strat_col"]
for train_idx, test_idx in split.split(housing, housing["strat_col"]):
    strat_train_set = housing.loc[train_idx].copy()
    strat_test_set  = housing.loc[test_idx].copy()

In [21]:
train_idx

array([20116, 12456,  6036, ...,  4591,  6866,  2367])

In [22]:
# Show distribution of income of the split data generated by the stratfication 
strat_test_set["strat_col"].value_counts() / len(strat_test_set)

strat_col
INLAND_Q1        0.249031
<1H OCEAN_Q1     0.245882
<1H OCEAN_Q2     0.172965
NEAR OCEAN_Q1    0.078488
INLAND_Q2        0.065165
NEAR BAY_Q1      0.064196
NEAR OCEAN_Q2    0.044089
NEAR BAY_Q2      0.040698
<1H OCEAN_Q3     0.019864
NEAR BAY_Q3      0.005087
NEAR OCEAN_Q3    0.004845
<1H OCEAN_Q4     0.003876
INLAND_Q3        0.002907
NEAR OCEAN_Q4    0.001211
NEAR BAY_Q4      0.001211
INLAND_Q4        0.000242
ISLAND_Q1        0.000242
Name: count, dtype: float64

In [23]:
# Compare with overal sample. Almost identical
housing["strat_col"].value_counts() / len(housing)

strat_col
INLAND_Q1        0.248934
<1H OCEAN_Q1     0.245979
<1H OCEAN_Q2     0.173014
NEAR OCEAN_Q1    0.078634
INLAND_Q2        0.065213
NEAR BAY_Q1      0.064147
NEAR OCEAN_Q2    0.043992
NEAR BAY_Q2      0.040698
<1H OCEAN_Q3     0.019816
NEAR BAY_Q3      0.004990
NEAR OCEAN_Q3    0.004942
<1H OCEAN_Q4     0.003828
INLAND_Q3        0.002907
NEAR OCEAN_Q4    0.001211
NEAR BAY_Q4      0.001114
INLAND_Q4        0.000339
ISLAND_Q1        0.000242
Name: count, dtype: float64

## STEP 7: Train and test Model 1(the one which uses stratified sampling)

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

#Now,I separate predictors and labels; drop helper columns from predictors
cols_to_drop = ["median_house_value", "income_quartile", "strat_col"]
housing_train = strat_train_set.drop(cols_to_drop, axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

#Next, I build preprocessing pipeline (median impute numeric, one-hot encode categorical)
cat_attribs = ["ocean_proximity"]
num_attribs = [c for c in housing_train.columns if c not in cat_attribs]

full_pipeline = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_attribs),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_attribs),
])

#I prepare training data and fit Linear Regression
housing_prepared = full_pipeline.fit_transform(housing_train)
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

#Finally, I prepare test set and compute test RMSE
housing_test = strat_test_set.drop(cols_to_drop, axis=1)
housing_test_labels = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(housing_test)

final_predictions = lin_reg.predict(X_test_prepared)
final_mse = mean_squared_error(housing_test_labels, final_predictions)
final_rmse_strat = np.sqrt(final_mse)
print("Stratified split - Test RMSE:", final_rmse_strat)

Stratified split - Test RMSE: 68695.7450369008


## STEP 8: Compare the results to a baseline model where the dataset is split into training and testing sets using a simple random split (without stratification)

In [25]:
from sklearn.model_selection import train_test_split

#I conduct random split on the full data (no stratification)
train_set_rand, test_set_rand = train_test_split(housing, test_size=0.2, random_state=42)

#Next, I separate predictors and labels; drop helper columns from predictors
housing_train_rand = train_set_rand.drop(cols_to_drop, axis=1)
housing_labels_rand = train_set_rand["median_house_value"].copy()

#And then build a new preprocessing pipeline for fairness and fit on random-train
cat_attribs = ["ocean_proximity"]
num_attribs = [c for c in housing_train_rand.columns if c not in cat_attribs]

full_pipeline_rand = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_attribs),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_attribs),
])

housing_prepared_rand = full_pipeline_rand.fit_transform(housing_train_rand)
lin_reg_rand = LinearRegression()
lin_reg_rand.fit(housing_prepared_rand, housing_labels_rand)

#Finally, I prepare random-test and compute test RMSE
housing_test_rand = test_set_rand.drop(cols_to_drop, axis=1)
housing_test_labels_rand = test_set_rand["median_house_value"].copy()
X_test_prepared_rand = full_pipeline_rand.transform(housing_test_rand)

final_predictions_rand = lin_reg_rand.predict(X_test_prepared_rand)
final_mse_rand = mean_squared_error(housing_test_labels_rand, final_predictions_rand)
final_rmse_rand = np.sqrt(final_mse_rand)
print("Random split - Test RMSE:", final_rmse_rand)

Random split - Test RMSE: 69791.2548561357


## STEP 9: Interpret results 

Using stratified sampling based on income quartiles and ocean proximity produced a test RMSE of about 68,700, which is slightly better than the 69,800 obtained with a simple random split. The difference is modest, but it demonstrates that stratified sampling helps ensure the training and test sets are representative of the population, leading to more reliable performance estimates.